# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import sqlalchemy as sa
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
import pickle

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/Arirammstein/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Arirammstein/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/Arirammstein/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Arirammstein/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# load data from database
engine = sa.create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('Messages', engine)

In [4]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize text 
    Input:
        text -> Text message to be tokenized
    Output:
        words -> List of tokens extracted from the input text
    """
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)
    
    # tokenize text
    tokens = word_tokenize(text)
    #Lemmatizer
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff1e90ad820>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
y_pred = pipeline.predict(X_test)

In [11]:
for i, j in enumerate(y.columns):
    print(j)
    print(classification_report(y_test[j], y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.74      0.25      0.38      1546
           1       0.80      0.97      0.88      4961
           2       0.45      0.11      0.17        47

    accuracy                           0.80      6554
   macro avg       0.67      0.44      0.48      6554
weighted avg       0.78      0.80      0.76      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5440
           1       0.89      0.44      0.59      1114

    accuracy                           0.90      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

/Users/Arirammstein/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5707
           1       0.57      0.01      0.03       847

    accuracy                           0.87      6554
   macro avg       0.72      0.51      0.48      6554
weighted avg       0.83      0.87      0.81      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6183
           1       0.33      0.00      0.01       371

    accuracy                           0.94      6554
   macro avg       0.64      0.50      0.49      6554
weighted avg       0.91      0.94      0.92      6554

transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6275
           1       0.74      0.07      0.13       279

    accuracy                           0.96      6554
   macro avg       0.85      0.54      0.55      6554
weighted avg       0.95      0.96      0.

### 6. Improve your model
Use grid search to find better parameters. 

In [12]:
parameters = {
        'clf__estimator__n_estimators': [1, 3, 10],
        'clf__estimator__max_leaf_nodes': [2, 3, 4],
    }

cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7ff1e90ad820>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_leaf_nodes': [2, 3, 4],
                         'clf__estimator__n_estimators': [1, 3, 10]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [13]:
y_pred = cv.predict(X_test)

In [14]:
for i, j in enumerate(y.columns):
    print(j)
    print(classification_report(y_test[j], y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1546
           1       0.76      1.00      0.86      4961
           2       0.00      0.00      0.00        47

    accuracy                           0.76      6554
   macro avg       0.25      0.33      0.29      6554
weighted avg       0.57      0.76      0.65      6554

request
              precision    recall  f1-score   support

           0       0.83      1.00      0.91      5440
           1       0.00      0.00      0.00      1114

    accuracy                           0.83      6554
   macro avg       0.42      0.50      0.45      6554
weighted avg       0.69      0.83      0.75      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

/Users/Arirammstein/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6275
           1       0.00      0.00      0.00       279

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.94      6554

buildings
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      6231
           1       0.00      0.00      0.00       323

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.90      0.95      0.93      6554

electricity
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6430
           1       0.00      0.00      0.00       124

    accuracy                           0.98      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.96      0.98      0.97      655

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [15]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [16]:
pipeline_oth = Pipeline([
    ('features', FeatureUnion([
        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
        ])),

    ('clf', RandomForestClassifier())
])

In [17]:

pipeline_oth.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x7ff1e90ad820>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())])),
                                                ('starting_verb',
                                                 StartingVerbExtractor())])),
                ('clf', RandomForestClassifier())])

In [18]:
y_pred = pipeline_oth.predict(X_test)

In [19]:
for i, j in enumerate(y.columns):
    print(j)
    print(classification_report(y_test[j], y_pred[:,i]))

related
              precision    recall  f1-score   support

           0       0.69      0.35      0.46      1546
           1       0.82      0.95      0.88      4961
           2       0.50      0.13      0.20        47

    accuracy                           0.80      6554
   macro avg       0.67      0.48      0.52      6554
weighted avg       0.79      0.80      0.78      6554

request
              precision    recall  f1-score   support

           0       0.89      0.99      0.94      5440
           1       0.90      0.43      0.58      1114

    accuracy                           0.89      6554
   macro avg       0.90      0.71      0.76      6554
weighted avg       0.90      0.89      0.88      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6527
           1       0.00      0.00      0.00        27

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      655

/Users/Arirammstein/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5707
           1       0.58      0.01      0.03       847

    accuracy                           0.87      6554
   macro avg       0.73      0.51      0.48      6554
weighted avg       0.83      0.87      0.81      6554

infrastructure_related
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      6183
           1       0.33      0.00      0.01       371

    accuracy                           0.94      6554
   macro avg       0.64      0.50      0.49      6554
weighted avg       0.91      0.94      0.92      6554

transport
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6275
           1       0.00      0.00      0.00       279

    accuracy                           0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.92      0.96      0.

In [22]:
### 9. Export your model as a pickle file

In [23]:
pickle.dumps('classifier.pkl')

b'\x80\x04\x95\x12\x00\x00\x00\x00\x00\x00\x00\x8c\x0eclassifier.pkl\x94.'

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.